In [ ]:

!pip install -q streamlit faiss-cpu sentence-transformers langchain langchain-community huggingface_hub pyngrok kaggle


In [ ]:

from google.colab import files
files.upload()  # 👉 Upload your kaggle.json here


In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sonalisingh1411/loan-approval-prediction
!unzip -o loan-approval-prediction.zip


In [ ]:

with open("utils.py", "w") as f:
    f.write("""import pandas as pd

def load_csv(file_obj):
    df = pd.read_csv(file_obj)
    df.dropna(inplace=True)
    return df

def rows_to_documents(df):
    return df.apply(lambda row: " | ".join(f"{col}: {val}" for col, val in row.items()), axis=1).tolist()
""")


In [ ]:

with open("retriever.py", "w") as f:
    f.write("""import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

class Retriever:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.index = None
        self.documents = []

    def fit(self, docs):
        self.documents = docs
        embeddings = self.model.encode(docs)
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings))

    def get_relevant_docs(self, query, k=5):
        query_emb = self.model.encode([query])
        _, I = self.index.search(np.array(query_emb), k)
        return [self.documents[i] for i in I[0]]
""")


In [ ]:

with open("qa_chain.py", "w") as f:
    f.write("""from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub

class QABot:
    def __init__(self, retriever):
        self.retriever = retriever
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.llm = HuggingFaceHub(
            repo_id="google/flan-t5-base",
            model_kwargs={"temperature": 0.1, "max_length": 512}
        )
        self.chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.retriever,
            memory=self.memory
        )

    def ask(self, query):
        return self.chain.run(query)
""")


In [ ]:
# ✅ Set Hugging Face API token BEFORE importing QABot
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_your_token_here"  # Replace with your actual token


In [ ]:

with open("app.py", "w") as f:
    f.write("""import streamlit as st
from utils import load_csv, rows_to_documents
from retriever import Retriever
from qa_chain import QABot

st.set_page_config(layout="wide")
st.title("📊 Loan Approval Chatbot (FREE MODE)")

uploaded = st.sidebar.file_uploader("Upload Loan CSV", type="csv")

if "bot" not in st.session_state:
    st.session_state.bot = None

if uploaded:
    df = load_csv(uploaded)
    docs = rows_to_documents(df)
    retr = Retriever()
    retr.fit(docs)
    st.session_state.bot = QABot(retriever=retr)
    st.success("✅ Chatbot is ready—ask any question!")

if st.session_state.bot:
    if "history" not in st.session_state:
        st.session_state.history = []
    query = st.text_input("Ask something:")
    if query:
        ans = st.session_state.bot.ask(query)
        st.session_state.history.append((query, ans))

    for q, a in reversed(st.session_state.history):
        st.markdown(f"**Q:** {q}")
        st.markdown(f"**A:** {a}")
""")


In [ ]:

!pkill -f streamlit
!streamlit run app.py &> logs.txt &
from pyngrok import ngrok
print("🔗 Public link:", ngrok.connect(8501))


In [ ]:

!tail -n 50 logs.txt
